In [1]:
import pandas as pd
import csv
import random 


FUNZIONE LEGGIDATI

In [2]:

def LeggiDati(filename):

    df=pd.read_csv(filename,header=0)

    corse=[]
    fermate=df.columns[2:].tolist()  #creo una lista con le fermate prendendo dalla seconda colonna in avanti

    for _,riga in df.iterrows():  #iterrows itera per indice e contenuto della riga, con i nomi delle colonne come chiavi.
        autista=riga.iloc[0]
        passeggeri=[]
        for dato in riga[2:]:
            if pd.isna(dato) or dato == '--': # il comando .isna() serve per capire se il valore è nullo
                passeggeri.append((0, 0))
            elif isinstance(dato, str) and '/' in dato: # il comando isintance(dato,str) controlla se dato è una stringa-> ritorna vero o falso
                saliti, scesi = map(int, dato.split('/')) # il comando map(funzione,iterabile) serve per applicare una funzione a ogni elemento di un iterabile
                passeggeri.append((saliti, scesi)) 
            elif isinstance(dato, str) and dato.startswith('-'):
                scesi = int(dato.rstrip('-'))
                passeggeri.append((0, scesi))
            else:
                    try:
                        saliti = int(dato)
                        passeggeri.append((saliti, 0))
                    except:
                        passeggeri.append((0, 0))

        corsa = {'autista': autista, 'passeggeri': passeggeri}
        corse.append(corsa)
    
    return df,corse,fermate




#for corsa in corse:
   # print(corsa)

FUNZIONE ESTRAI_SALITI_SCESI

In [3]:
def estrai_saliti_scesi(val):
    if val == '--' or pd.isna(val):
        return (0, 0)
    elif '/' in val:
        saliti, scesi = val.split('/')
        return (int(saliti), -int(scesi))
    elif val.startswith('-'):
        return (0, -int(val.strip('-')))
    else:
        try:
            return (int(val), 0)
        except:
            return (0, 0)

In [4]:
def estrai_saliti_scesi_cla(val):
    if val == '--' or pd.isna(val):
        return ('0/0', 0,0)
    elif '/' in val:
        saliti, scesi = val.split('/')
        return (val,int(saliti), -int(scesi))
    elif val.startswith('-'):
        return (f"0/{int(val.strip('-'))}",0, -int(val.strip('-')))
    else:
        try:
            return (f'{int(val)}/0',int(val), 0)
        except:
            return ('0/0',0, 0)

In [5]:
df_ae=LeggiDati('dati_aeroporto_torino.csv')[0]


#creare tabella ID time fermata con colonne scesi saliti

df_ae_melt=pd.melt(df_ae,id_vars=['ID', 'Time'],var_name='Fermata_ID',value_name='Passeggeri')  #idvars =identificativo di oggni riga    varname=come chiamo la nuova colonna che prende i nomi delle colonne passate(eccetto quelle identificative) 
df_ae_melt.rename(columns={'ID':'Autista_ID'}, inplace=True)


df_ae_melt['Passeggeri2'] = df_ae_melt['Passeggeri'].astype(str).str.strip() #rimuovo spazi e mi assicuro che siano stringhe, così le converto con la prossima funzione in interi
df_ae_melt[['Passeggeri2','Saliti', 'Scesi']] = df_ae_melt['Passeggeri2'].apply(estrai_saliti_scesi_cla).apply(pd.Series) #applico la funzione estrai_saliti_scesi a tutte le righe



df_ae_melt['Totale_Passeggeri']=df_ae_melt['Saliti'] - (df_ae_melt['Scesi'])
#display(df_ae_melt.tail(50))
pd.set_option('display.max_rows', None)
#display(df_ae_melt.loc[df_ae_melt['Saliti']==0])
pd.reset_option('display.max_rows')








FileNotFoundError: [Errno 2] No such file or directory: 'dati_aeroporto_torino.csv'

FUNZIONE LEGGINOMI

In [6]:
def LeggiNomi(filename): 
    df=pd.read_csv(filename, sep=':',header=None, names=['Code','Name'])
    dict_nomi={}
    for _,riga in df.iterrows():
        dict_nomi[riga.iloc[0]]=riga.iloc[1]
    #print(dict_nomi)

    return df, dict_nomi

JOIN(merge) TRA DF_DATABASE E DF_AE_MELT

In [7]:

df_db= LeggiNomi('database.txt')[0]
df_db.rename(columns={'Code':'Autista_ID'}, inplace=True)
#display(df_db.head())

df_joined=df_ae_melt.merge(df_db, how='left', on='Autista_ID')
df_joined=df_joined[['Autista_ID','Name','Time','Fermata_ID','Passeggeri','Saliti','Scesi','Totale_Passeggeri']] #ho spostato la colonna 'Name' affianco a quella di 'Autista_ID'
df_joined.rename(columns={'Name':'Autista_Name'}, inplace=True)
#display(df_joined.sort_values(by='Totale_Passeggeri'))
display(df_joined)
#display(df_joined.iloc[50:100])


SalitiRaccolti=df_joined.groupby('Fermata_ID')['Saliti'].sum()
display(SalitiRaccolti)
ScesiRaccolti=df_joined.groupby('Fermata_ID')['Scesi'].sum()
display(ScesiRaccolti)

max_saliti=SalitiRaccolti.max()
#display(max_saliti)
max_scesi=-(ScesiRaccolti.min())

indice_fermata_max= SalitiRaccolti.idxmax()
#display(indice_fermata_max)
indice_fermata_min=ScesiRaccolti.idxmin()

#MaxSaliti=df_fermateRaccolte['Saliti'].max()
#display(MaxSaliti)
print(f'La fermata con più saliti è {indice_fermata_max} con {max_saliti} passeggeri saliti ')
print(f'La fermata con più scesi è {indice_fermata_min} con {max_scesi} passeggeri scesii')

provaCommit=5


'''probelmi da risolvere: 
- se ci sono solo passeggeri che scendono alla fermata, nella colonna passeggeri mi da il numero col - quando dovrebbe essere col +
- nel print al posto del codice della fermata devo mettere il nome della fermata
'''

FileNotFoundError: [Errno 2] No such file or directory: 'database.txt'

FUNZIONE CALCOLA PASSEGGERI PER AUTISTA
l'ho lasciata uguale a quella originale, non tocca i dataframe

In [ ]:
def calcolaPasseggeriXAutista(corse):
    fermate=[]

    saliti={}
    for corsa in corse:
        tot=0
        for tupla in corsa['passeggeri']:
            tot+=tupla[0]

        autista=corsa['autista']
        if autista in saliti:
            saliti[autista]+=tot
        else:
            saliti[autista]=tot

    return saliti


FUNZIONE MAIN

In [ ]:
def main():
    
    df_ae, corse,fermate=LeggiDati('dati_aeroporto_torino.csv')
    df_db,nomi=LeggiNomi('database.txt')

    display(df_db)

    
    salitiMax=0
    scesiMax=0
    for fermata in range(len(fermate)):
        saliti=0
        scesi=0
        for corsa in corse:
            saliti+=corsa['passeggeri'][fermata][0]
            scesi+=corsa['passeggeri'][fermata][1]
        if saliti>salitiMax:
            salitiMax=saliti
            fermataSalitiMax=fermate[fermata]
        if scesi>scesiMax:
            scesiMax=scesi
            fermataScesiMAx=fermate[fermata]

    print(f'La fermata con il massimo di saliti {salitiMax} è {nomi[fermataSalitiMax]}')
    print(f'La fermata con il massimo di scesi {scesiMax} è {nomi[fermataScesiMAx]}')



    passeggerixAutista = calcolaPasseggeriXAutista(corse)
    print(passeggerixAutista)
    
main()

,Code,Name
0,A002,Sebastiano Canali
1,A003,Giacobbe Comboni
2,A004,Lolita Crisafulli
3,A006,Silvio Cabibbo
4,A008,Fausto Papafava
5,A010,Barbara Bush
6,A012,Eliana Capuana
7,A014,Verim Duran
8,A015,Ivo Cendron
9,A020,Fuzuli Soylu


La fermata con il massimo di saliti 1626 è  Aeroporto
La fermata con il massimo di scesi 39 è  Caselle 2
{'A012': 50, 'A003': 49, 'A041': 46, 'A240': 52, 'A119': 40, 'A015': 37, 'A154': 32, 'A008': 43, 'A212': 12, 'A021': 22, 'A022': 16, 'A109': 15, 'A200': 21, 'A010': 16, 'A139': 11, 'A006': 10}
